# column을 drop시킬 corr값(threshold)을 변경해가면서 테스트해본다.

In [17]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [18]:
# base_path = './preprocessed data/'

# train_err = pd.read_csv(base_path + 'new_train_err.csv', parse_dates=['time'])
# train_quality = pd.read_csv(base_path + 'new_train_quality.csv')
# train_problem = pd.read_csv(base_path + 'new_train_problem.csv', parse_dates=['time'])
# test_err = pd.read_csv(base_path + 'new_test_err.csv', parse_dates=['time'])
# test_quality = pd.read_csv(base_path + 'new_test_quality.csv', parse_dates=['time'])
# submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")
train = pd.read_csv('./training set/train_has_duplicate.csv')
train_problem = pd.read_csv('./preprocessed data/new_train_problem.csv', parse_dates=['time'])

case1. threshold=0.8

case2. threshold=0.9

In [20]:
# 중복정보가 있는 column 제거
def remove_collinearity(X, threshold):
    """
    X : feature matrix
    threshold : 다중공선성을 제거할 column을 고르는 기준 값. [0, 1]
    """
    
    corr = X.corr()
    candidate_cols = []
    
    for x in corr.iterrows():
        idx, row = x[0], x[1] # decoupling tuple
        if idx in candidate_cols:
            continue
        candidates = row[row > threshold].index[1:]

        # 자기 자신을 제외하고 threshold를 넘는 column이 있다면 제거 할 column이다.
        if len(candidates) != 0:
            for col in candidates:
                candidate_cols.append(col)           
    
    return candidate_cols

In [20]:
# case0. threshold = 0.7
# correlated_features = remove_collinearity(dfs, 0.7)
# print(correlated_features)
# dfs = dfs.drop(columns=correlated_features)
# len(dfs.columns)

['errtype_4', 'errtype_num', 'errtype_weight', 'errcode_weight', 'fwver_04.22.1750', 'fwver_04.22.1778', 'fwver_04.16.3553', 'errtype_22', 'errtype_23', 'fwver_04.33.1185', 'fwver_04.33.1261', 'fwver_05.15.2138', 'errtype_4', 'fwver_03.11.1167', 'errtype_27', 'errtype_28', 'fwver_04.82.1684', 'fwver_04.82.1778', 'fwver_10', 'fwver_05.66.3237', 'fwver_04.73.2237', 'errtype_10', 'errtype_11', 'errtype_12', 'errtype_7', 'errtype_16', 'errtype_20', 'errtype_20', 'errtype_21', 'errtype_35', 'errtype_37', 'multiple_fwver', 'quality_2', 'quality_9', 'quality_10', 'quality_7']


70

In [21]:
# case1. threshold = 0.8
correlated_features = remove_collinearity(train, 0.8)
print(correlated_features)
train_case1 = train.drop(columns=correlated_features)
len(train_case1.columns)

['errtype_num', 'errtype_weight', 'errcode_weight', 'fwver_04.22.1750', 'fwver_04.16.3553', 'fwver_04.33.1185', 'fwver_04.33.1261', 'fwver_05.15.2138', 'errtype_4', 'fwver_03.11.1167', 'errtype_27', 'errtype_28', 'fwver_04.82.1684', 'fwver_04.82.1778', 'fwver_10', 'fwver_05.66.3237', 'fwver_04.73.2237', 'errtype_7', 'errtype_11', 'errtype_12', 'errtype_16', 'errtype_21', 'errtype_23', 'errtype_37', 'multiple_fwver', 'quality_2', 'quality_9', 'quality_10', 'quality_7']


75

In [22]:
# case2. threshold = 0.9
correlated_features = remove_collinearity(train, 0.9)
print(correlated_features)
train_case2 = train.drop(columns=correlated_features)
len(train_case2.columns)

['errtype_num', 'errtype_weight', 'errcode_weight', 'fwver_04.22.1750', 'fwver_04.16.3553', 'fwver_04.33.1261', 'fwver_05.15.2138', 'errtype_4', 'fwver_03.11.1167', 'errtype_27', 'errtype_28', 'fwver_04.82.1684', 'fwver_10', 'fwver_05.66.3237', 'fwver_04.73.2237', 'errtype_7', 'errtype_11', 'errtype_12', 'errtype_16', 'errtype_21', 'errtype_23', 'errtype_37', 'quality_9', 'quality_9', 'quality_10', 'quality_7']


79